<a href="https://colab.research.google.com/github/Sovanno/NLP/blob/main/LAB2_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка датасета

In [1]:
import pandas as pd

Загрузка данных

In [2]:
data = pd.read_csv('Petitions.csv')
data

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
59884,3128111,прошу закрасить граффити,Благоустройство
59885,3276713,Прошу вас отремонтировать пешеходную дорожку,Благоустройство
59886,3274663,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...
59887,3359308,Очень гремит на ветру металлическая часть окна...,Кровля


Удаляем пустые строки

In [5]:
data = data.dropna()
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43336 entries, 0 to 59888
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   public_petition_text  43336 non-null  object
 1   reason_category       43336 non-null  object
dtypes: object(2)
memory usage: 1015.7+ KB


Фильтрация данных (берем 30000 примеров)

In [6]:
sample_size = 30000
data_sample = data.sample(n=sample_size, random_state=42)
data_sample

,public_petition_text,reason_category
4093,"улица Геологическая 75, рядом с корпусом 6, на...",Благоустройство
42921,"Здравствуйте.\nВ нашес доме, по адресу: Короле...",Санитарное состояние
20237,В новом лифте 1 парадной освещение не проработ...,Содержание МКД
7948,"Добрый день, уважаемый исполнитель! Выявлены н...",Благоустройство
22523,вся дорога в ОГРОМНЫХ ямищах,Благоустройство
...,...,...
28482,"Наличие сухих высоких деревьев, опасный наклон...",Благоустройство
45951,Вырезан кусок асвальта пешеходного тратуара,Благоустройство
8136,Мешок лежит на газоне у остановки не первый день,Благоустройство
26,Очистить,Благоустройство


# Предварительная обработка

In [14]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

Функция предварительной обработки текста

In [11]:
!pip install num2words
from num2words import num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 4.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ca610b0ea0ab3b1fd4ed67bc97910bcc52b5cddbc785fcf138df680486eab726
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [12]:
def preprocess_text(text):
    text = text.lower()
    http = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    text = re.sub(http, "", text)
    text = re.sub('\S+@\S+\.\S+', '', text)
    text = re.sub('\+?[1-9][0-9]{10}', '', text)
    text = re.sub(r"\d+", lambda match: " " + num2words(int(match.group(0)), lang="ru") + " ", text)
    text = re.sub(r"\bд\b", "дом", text)
    text = re.sub(r"\bкв\b", "квартира", text)
    text = re.sub(r"\bул\b", "улица", text)
    text = re.sub(r"\bк\b", "квартира", text)
    text = re.sub('[^\s^\w]+', '', text)
    text = re.sub(r"[^а-яА-ЯёЁ\s]", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = word_tokenize(text)

    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [15]:
data_sample['public_petition_text'] = data_sample['public_petition_text'].apply(preprocess_text)
data_sample

,public_petition_text,reason_category
4093,"[улица, геологическая, семьдесят, пять, рядом,...",Благоустройство
42921,"[здравствуйтев, нашес, доме, адресу, королева,...",Санитарное состояние
20237,"[новом, лифте, парадной, освещение, проработал...",Содержание МКД
7948,"[добрый, день, уважаемый, исполнитель, выявлен...",Благоустройство
22523,"[вся, дорога, огромных, ямищах]",Благоустройство
...,...,...
28482,"[наличие, сухих, высоких, деревьев, опасный, н...",Благоустройство
45951,"[вырезан, кусок, асвальта, пешеходного, тратуара]",Благоустройство
8136,"[мешок, лежит, газоне, остановки, первый, день]",Благоустройство
26,[очистить],Благоустройство


In [16]:
data_sample['reason_category'].value_counts()

,count
reason_category,
Благоустройство,15151
Содержание МКД,9701
Фасад,849
Повреждения или неисправность элементов уличной инфраструктуры,677
Нарушение правил пользования общим имуществом,656
Незаконная информационная и (или) рекламная конструкция,612
Водоснабжение,544
Кровля,501
Санитарное состояние,272


Нужно сократить количество классов


1.   Благоустройство и Содержание МКД:
   - Объединить в класс Управление территорией.
   - Т.к. оба класса связаны с поддержанием и улучшением состояния территории и объектов, что подразумевает управление общими ресурсами.
2.   Фасад, Кровля и Подвалы:
   - Объединить в класс Элементы здания.
   - Т.к. все эти классы относятся к элементам конструкции здания, их состоянию и необходимости ремонтов.
3.   Водоснабжение и Водоотведение:
   - Объединить в класс Водоснабжения.
   - Т.к. оба класса относятся к водным системам и их функциональности.
4.   Повреждения или неисправность элементов уличной инфраструктуры и Нарушение правил пользования общим имуществом:
   - Объединить в класс Нарушения.
   - Т.к. оба класса касаются вопросов нарушения эксплуатации и состояния общего имущества.
5.   Санитарное состояние и Нарушение порядка пользования общим имуществом:
   - Объединить в класс Гигиена и порядок.
   - Т.к. оба класса охватывают вопросы санитарии и использования общественных ресурсов, что напрямую влияет на комфорт и здоровье жителей.
6.   Незаконная информационная и (или) рекламная конструкция и Состояние рекламных или информационных конструкций:
   - Объединить в класс Реклама.
   - Т.к. оба класса связаны с вопросами, касающимися рекламной активности.
7.   Центральное отопление:
   - Оставлен как отдельный класс, так как имеет специфические технические особенности и отличается от других классов.
8.   Незаконная реализация товаров с торгового оборудования:
   - Оставлен как отдельный класс, так как касается вопросов незаконной торговли, что является самостоятельной проблемой.

In [18]:
replace ={'Благоустройство' : 'Управление территорией',
              'Содержание МКД' : 'Управление территорией',
              'Фасад' : 'Элементы здания',
              'Кровля' : 'Элементы здания',
              'Подвалы' : 'Элементы здания',
              'Водоотведение' : 'Водоснабжение',
              'Повреждения или неисправность элементов уличной инфраструктуры' : 'Нарушения',
              'Нарушение правил пользования общим имуществом' : 'Нарушения',
              'Санитарное состояние' : 'Гигиена и порядок',
              'Нарушение порядка пользования общим имуществом' : 'Гигиена и порядок',
              'Незаконная информационная и (или) рекламная конструкция' : 'Реклама',
              'Состояние рекламных или информационных конструкций' : 'Реклама',
              'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)' : 'Незаконная реализация товаров с торгового оборудования'
}
data_sample['reason_category'].replace(replace, inplace=True)
data_sample['reason_category'].value_counts()

<ipython-input-18-72cb5aad3059>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_sample['reason_category'].replace(replace, inplace=True)


,count
reason_category,
Управление территорией,24852
Элементы здания,1511
Нарушения,1333
Реклама,821
Водоснабжение,710
Гигиена и порядок,451
Центральное отопление,196
Незаконная реализация товаров с торгового оборудования,126


# Векторизация документов

In [19]:
from gensim.models import Word2Vec

In [20]:
t_date = data_sample['public_petition_text']
t_date

,public_petition_text
4093,"[улица, геологическая, семьдесят, пять, рядом,..."
42921,"[здравствуйтев, нашес, доме, адресу, королева,..."
20237,"[новом, лифте, парадной, освещение, проработал..."
7948,"[добрый, день, уважаемый, исполнитель, выявлен..."
22523,"[вся, дорога, огромных, ямищах]"
...,...
28482,"[наличие, сухих, высоких, деревьев, опасный, н..."
45951,"[вырезан, кусок, асвальта, пешеходного, тратуара]"
8136,"[мешок, лежит, газоне, остановки, первый, день]"
26,[очистить]


In [21]:
model = Word2Vec(sentences=t_date, vector_size=128, window=3, min_count=1, workers=4, epochs=10)

In [22]:
model.wv['дорога'], model.wv['дорога'].shape

(array([-0.23763542, -0.7577411 , -0.18280603,  0.61395013,  0.00706783,
        -0.7741954 ,  0.25513065, -0.4401248 ,  0.2955347 ,  0.6544291 ,
         0.16187415,  0.16951105,  0.22451372, -0.07800376,  0.35375977,
         0.9097117 , -0.27131447, -0.04277654,  0.60673666,  0.7269181 ,
         0.30951926, -0.13065386, -0.5112217 , -1.4993914 ,  0.11150774,
         0.3474071 , -1.38215   ,  0.66807175,  0.38538095, -0.67140675,
        -0.04271655, -0.2346795 ,  0.78620046,  0.3789245 ,  0.51663417,
         0.31087515,  0.62276334,  0.14926253, -0.04767799,  0.32548198,
        -1.2493602 ,  0.37124318,  0.42347252, -0.40721786,  0.49769747,
        -0.54701024, -0.6735021 , -0.1610473 ,  0.33854133,  0.25259948,
        -0.02829318,  0.13990082, -0.05547702,  0.3149147 ,  0.0257468 ,
         0.44494072, -0.15840542, -0.20468804, -0.6404213 ,  0.5232349 ,
        -0.27266335, -0.7151725 ,  0.16308081,  0.3177811 ,  0.14787275,
         0.08046057,  0.582979  , -0.2759862 ,  0.8

In [24]:
model.wv.most_similar('улица', topn=5)

[('революции', 0.9385172128677368),
 ('наб', 0.9383726119995117),
 ('трудящихся', 0.9354119300842285),
 ('бульвар', 0.9311248064041138),
 ('огнева', 0.9274898171424866)]

In [25]:
model.save('mymodelw2v.bin')

# Кодировка целевого признака и разделение данных

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [27]:
labels = data_sample['reason_category']
labels

,reason_category
4093,Управление территорией
42921,Гигиена и порядок
20237,Управление территорией
7948,Управление территорией
22523,Управление территорией
...,...
28482,Управление территорией
45951,Управление территорией
8136,Управление территорией
26,Управление территорией


In [28]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels

array([5, 1, 5, ..., 5, 5, 1])

In [29]:
def vectorize_text(tokens):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return sum(vectors) / len(vectors)
    else:
        return [0] * model.vector_size

data_vec = data_sample['public_petition_text'].apply(vectorize_text)
data_vec

,public_petition_text
4093,"[-0.010980215, -0.38650647, 0.17566991, 0.8653..."
42921,"[0.3493809, -0.36370617, 0.4299601, 0.48645157..."
20237,"[0.7603574, -0.6895664, 0.05680578, 1.1655427,..."
7948,"[0.77892035, -0.49652794, 0.17292763, 0.202112..."
22523,"[0.075929195, -0.38120955, -0.0017583163, 0.21..."
...,...
28482,"[0.04873977, -0.26060942, 0.50070196, 0.569148..."
45951,"[-0.028161686, -0.31256738, -0.018116465, 0.05..."
8136,"[0.42477176, -0.2742916, -0.015448749, 0.44211..."
26,"[0.9979334, -0.8964651, -0.68364704, 0.538392,..."


In [30]:
len(encoded_labels)

30000

In [31]:
X_train, X_test, y_train, y_test = train_test_split(data_vec, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels)

# Обучение Рекуррентных нейронных сетей для задачи классификации

In [38]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

In [52]:
X_train

,public_petition_text
39569,"[0.4206555, -0.34754646, 0.09521072, 1.863, 0...."
10394,"[0.21383004, -0.33692542, -0.079687245, 1.3285..."
38901,"[0.69837034, -0.47685155, 0.3233039, -0.119972..."
5558,"[0.2865792, -0.1320491, 0.54839665, 0.13531992..."
17549,"[0.36377028, -0.64167416, 0.15143153, 0.279739..."
...,...
35571,"[-0.610381, -0.81938374, 0.7114699, 0.28210902..."
32527,"[-0.19058746, -0.4835949, -0.009201312, 0.3344..."
12529,"[0.2717864, -0.46200114, 0.20938018, 0.6728012..."
25964,"[-0.007292069, -0.22208993, 0.15136757, 0.2082..."


In [54]:
X_train = np.array(list(X_train))
X_test = np.array(list(X_test))

In [56]:
len(X_train)

24000

In [60]:
X_train_t = torch.tensor(X_train)
y_train_t = torch.tensor(y_train)
X_test_t = torch.tensor(X_test)
y_test_t = torch.tensor(y_test)

In [61]:
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

In [62]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [63]:
import torch.nn as nn
import torch.optim as optim

In [73]:
input_size = X_train.shape[1]
hidden_size = 256
num_classes = len(np.unique(y_train))
num_epochs = 20
learning_rate = 0.001

In [65]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [71]:
def train_test_model(model, train_loader, test_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

In [74]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNClassifier, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

rnn_model = RNNClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=learning_rate)

train_test_model(rnn_model, train_loader, test_loader, criterion, optimizer, num_epochs)

Epoch 1/20, Loss: 0.5774
Epoch 2/20, Loss: 0.4650
Epoch 3/20, Loss: 0.4366
Epoch 4/20, Loss: 0.4176
Epoch 5/20, Loss: 0.4032
Epoch 6/20, Loss: 0.3924
Epoch 7/20, Loss: 0.3817
Epoch 8/20, Loss: 0.3728
Epoch 9/20, Loss: 0.3644
Epoch 10/20, Loss: 0.3584
Epoch 11/20, Loss: 0.3534
Epoch 12/20, Loss: 0.3484
Epoch 13/20, Loss: 0.3407
Epoch 14/20, Loss: 0.3387
Epoch 15/20, Loss: 0.3366
Epoch 16/20, Loss: 0.3321
Epoch 17/20, Loss: 0.3277
Epoch 18/20, Loss: 0.3264
Epoch 19/20, Loss: 0.3242
Epoch 20/20, Loss: 0.3228
Accuracy: 0.8945

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.75      0.81       142
           1       0.58      0.12      0.20        90
           2       0.77      0.60      0.67       267
           3       0.67      0.24      0.35        25
           4       0.76      0.38      0.51       164
           5       0.91      0.98      0.95      4971
           6       0.83      0.13      0.22        39
           7   

In [75]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

lstm_model = LSTMClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)

train_test_model(lstm_model, train_loader, test_loader, criterion, optimizer, num_epochs)

Epoch 1/20, Loss: 0.6011
Epoch 2/20, Loss: 0.4594
Epoch 3/20, Loss: 0.4164
Epoch 4/20, Loss: 0.3882
Epoch 5/20, Loss: 0.3680
Epoch 6/20, Loss: 0.3560
Epoch 7/20, Loss: 0.3435
Epoch 8/20, Loss: 0.3330
Epoch 9/20, Loss: 0.3270
Epoch 10/20, Loss: 0.3186
Epoch 11/20, Loss: 0.3121
Epoch 12/20, Loss: 0.3086
Epoch 13/20, Loss: 0.3009
Epoch 14/20, Loss: 0.2983
Epoch 15/20, Loss: 0.2941
Epoch 16/20, Loss: 0.2882
Epoch 17/20, Loss: 0.2856
Epoch 18/20, Loss: 0.2830
Epoch 19/20, Loss: 0.2791
Epoch 20/20, Loss: 0.2764
Accuracy: 0.9031666666666667

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82       142
           1       0.59      0.26      0.36        90
           2       0.82      0.64      0.72       267
           3       0.60      0.24      0.34        25
           4       0.86      0.54      0.66       164
           5       0.91      0.99      0.95      4971
           6       0.60      0.08      0.14        39
   

In [76]:
class GRUClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(GRUClassifier, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

gru_model = GRUClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gru_model.parameters(), lr=learning_rate)

train_test_model(gru_model, train_loader, test_loader, criterion, optimizer, num_epochs)

Epoch 1/20, Loss: 0.5608
Epoch 2/20, Loss: 0.4357
Epoch 3/20, Loss: 0.3999
Epoch 4/20, Loss: 0.3792
Epoch 5/20, Loss: 0.3629
Epoch 6/20, Loss: 0.3479
Epoch 7/20, Loss: 0.3411
Epoch 8/20, Loss: 0.3309
Epoch 9/20, Loss: 0.3257
Epoch 10/20, Loss: 0.3188
Epoch 11/20, Loss: 0.3118
Epoch 12/20, Loss: 0.3073
Epoch 13/20, Loss: 0.3057
Epoch 14/20, Loss: 0.3020
Epoch 15/20, Loss: 0.2976
Epoch 16/20, Loss: 0.2929
Epoch 17/20, Loss: 0.2906
Epoch 18/20, Loss: 0.2872
Epoch 19/20, Loss: 0.2849
Epoch 20/20, Loss: 0.2811
Accuracy: 0.897

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.80      0.82       142
           1       0.74      0.22      0.34        90
           2       0.86      0.49      0.63       267
           3       0.78      0.28      0.41        25
           4       0.75      0.57      0.65       164
           5       0.91      0.98      0.95      4971
           6       0.57      0.10      0.17        39
           7    

Поиграемся с готовыми моделями

In [80]:
text = "Прошу вас отремонтировать пешеходную дорожку"

tokens = preprocess_text(text)
input_vector = vectorize_text(tokens)
input_vector = np.array(list(input_vector))
input_vector = torch.tensor(input_vector)
input_tensor = input_vector.unsqueeze(0)

In [81]:
rnn_model.eval()

with torch.no_grad():
    output = rnn_model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Предсказанный класс: {predicted_class}")

predicted_label = label_encoder.inverse_transform([predicted_class])[0]
print(f"Класс текста: {predicted_label}")

Предсказанный класс: 5
Класс текста: Управление территорией


In [82]:
lstm_model.eval()

with torch.no_grad():
    output = lstm_model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Предсказанный класс: {predicted_class}")

predicted_label = label_encoder.inverse_transform([predicted_class])[0]
print(f"Класс текста: {predicted_label}")

Предсказанный класс: 5
Класс текста: Управление территорией


In [83]:
gru_model.eval()

with torch.no_grad():
    output = gru_model(input_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Предсказанный класс: {predicted_class}")

predicted_label = label_encoder.inverse_transform([predicted_class])[0]
print(f"Класс текста: {predicted_label}")

Предсказанный класс: 5
Класс текста: Управление территорией
